In [1]:
from deuling_dqn import *
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
filename_inputs = [
    'STB',
 'SAB',
 'VJC',
 'VRE',
 'VHM',
 'ACB',
 'GVR',
 'VNM',
 'SSB',
 'FPT',
 'BID',
#  'POW',
 'MBB',
 'PLX',
 'GAS',
 'HPG',
 'HDB',
 'MWG',
 'VCB',
 'SSI',
 'TCB',
 'SHB',
 'CTG',
 'BVH',
 'VIC',
 'VIB',
 'TPB',
 'MSN',
 'VPB',
 'BCM'
 ]


In [3]:
def main(filename_input):
    try:
        dataset = pd.read_csv(f'../RL_data/{filename_input}VPIN.csv', index_col=0).dropna()
    except FileNotFoundError:
        print("Data file not found. Please ensure the VCBVPIN.csv file is in the 'data' directory.")
        return

    os.makedirs('data', exist_ok=True)
    os.makedirs(f'../models/deulindqn/{filename_input}', exist_ok=True)

    price_vpin_data = dataset[["Price", "VPIN"]].dropna().values
    train_size = int(len(price_vpin_data) * 0.8)
    train_data = price_vpin_data[:train_size]
    test_data = price_vpin_data[train_size:]
    scaler = MinMaxScaler()
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)

    window_size, batch_size, episodes = 10, 32, 5
    model_name = f"../models/deulindqn/{filename_input}/model_sp500.pth"

    def evaluate_model(stock_data, save_plot):
        model_dir = f'../models/deulindqn/{filename_input}'
        model_files = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
        if not model_files:
            print("No model files found in 'models/' directory.")
            return

        print("Available model files:")
        for i, f in enumerate(model_files):
            print(f"{i + 1}. {f}")

        
        model_choices=["1", "2", "3", "4", "5", "6"]
        for model_choice in model_choices:
            if not model_choice.isdigit() or not (1 <= int(model_choice) <= len(model_files)):
                print("Invalid choice.")
                return
            chosen_model = os.path.join(model_dir, model_files[int(model_choice) - 1])
            agent = Agent(window_size * 2, is_eval=True, model_name=chosen_model)
            data_length = len(stock_data) - 1
            state = getState(stock_data, 0, window_size + 1)
            total_profit, agent.inventory = 0, []
            states_buy, states_sell = [], []

            print(f"Evaluating on {data_length} data points")
            for t in range(data_length):
                action = agent.act(state)
                next_state = getState(stock_data, t + 1, window_size + 1) if t + 1 < len(stock_data) else state

                if action == 1:
                    agent.inventory.append(stock_data[t])
                    states_buy.append(t)
                    print(f"Step {t}: Buy: {formatPrice(stock_data[t][0], scaler)}")
                elif action == 2 and agent.inventory:
                    bought_price = agent.inventory.pop(0)
                    sell_price = formatPrice(stock_data[t][0], scaler)
                    buy_price = formatPrice(bought_price[0], scaler)
                    profit = float(sell_price.replace("$", "").replace("-", "")) - float(buy_price.replace("$", "").replace("-", ""))
                    total_profit += profit
                    states_sell.append(t)
                    print(f"Step {t}: Sell: {sell_price} | Profit: {profit:.2f}")

                state = next_state

            print("------------------------------------------")
            print(f"Total Profit: ${total_profit:.2f}")
            print(f"Total Buy actions: {len(states_buy)}")
            print(f"Total Sell actions: {len(states_sell)}")
            print("------------------------------------------")

            plot_filename = f"trading_behavior_{os.path.basename(chosen_model).replace('.pth', '')}.png"
            plot_behavior(stock_data, states_buy, states_sell, total_profit, scaler, save_plot=save_plot, filename=plot_filename)

    save_plot=True
    evaluate_model(test_data, save_plot)



In [4]:
if __name__ == "__main__":
    for filename_input in filename_inputs:
        try:
            print(filename_input)
            main(filename_input)
        except:
            pass

STB
Available model files:
1. model_sp500_epoch_5.pth
2. model_sp500_epoch_4.pth
3. model_sp500.pth
4. model_sp500_epoch_1.pth
5. model_sp500_epoch_3.pth
6. model_sp500_epoch_2.pth
Evaluating on 530 data points
Step 19: Buy: $34.84
Step 20: Buy: $34.84
Step 25: Sell: $34.97 | Profit: 0.13
Step 26: Sell: $34.97 | Profit: 0.13
Step 67: Buy: $35.63
Step 71: Buy: $35.53
Step 80: Sell: $35.55 | Profit: -0.08
Step 81: Sell: $35.55 | Profit: 0.02
Step 101: Buy: $35.50
Step 102: Buy: $35.46
Step 103: Buy: $35.45
Step 104: Buy: $35.43
Step 105: Buy: $35.40
Step 106: Buy: $35.40
Step 107: Buy: $35.40
Step 108: Buy: $35.40
Step 116: Sell: $35.44 | Profit: -0.06
Step 117: Sell: $35.49 | Profit: 0.03
Step 118: Sell: $35.50 | Profit: 0.05
Step 119: Sell: $35.49 | Profit: 0.06
Step 120: Sell: $35.50 | Profit: 0.10
Step 121: Sell: $35.47 | Profit: 0.07
Step 122: Buy: $35.33
Step 125: Buy: $35.15
Step 126: Buy: $35.08
Step 127: Buy: $35.08
Step 128: Buy: $35.17
Step 129: Buy: $35.16
Step 130: Buy: $35.